In [1]:
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    root = '/content/drive/MyDrive/colab/sat-images/'
    device = 'cuda'
    !wget -O sat.zip https://github.com/lacykaltgr/satellite-image-segmentation/archive/refs/heads/laci420.zip
    !unzip sat.zip
    !cp -r satellite-image-segmentation-laci420/src .
else:
    root = '/Users/laszlofreund/code/ai/satellite-image-segmentation/data/'
    device = 'mps'
    import sys
    import os
    # Add the parent directory to the Python path
    parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))
    sys.path.append(parent_directory)

In [2]:
import torch
from torch.utils.data import DataLoader
from src.dataset import XYDataset

dataset = XYDataset(root='data/')
train_dataset = dataset[:int(0.9*len(dataset))]
val_dataset = dataset[int(0.9*len(dataset)):]

batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [3]:
import torch
from torch import nn
from unet import UNet
from utils import EarlyStopper
from src.utils import binary_to_rgb

config = dict(
    in_channels=3,
    out_channels=9,
    hiddens=[64, 128, 256],
    dropouts=[0, 0, 0.3, 0.3, 0.3],  # hiddens
    maxpools=2,  # hiddens - 1
    kernel_sizes=3,  # 2*hiddens + 3*hiddens + 2
    paddings='same',  # 2*hiddens + 3*hiddens + 2
    strides=1,  # 2*hiddens + 3*hiddens
    criterion=nn.CrossEntropyLoss,
    output_activation_function=nn.Softmax,
    activation=nn.ReLU,
    post_process=binary_to_rgb,
    dimensions=2,
    device='cuda'
)
unet = UNet(**config)

In [4]:
es = EarlyStopper(patience=10)
unet.train_model(train_loader, val_loader, es, num_epochs=1000, learning_rate=0.0001, device=device)

RuntimeError: MPS backend out of memory (MPS allocated: 15.73 GB, other allocations: 2.35 GB, max allowed: 18.13 GB). Tried to allocate 256.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [6]:
predictions, results = unet.predict(val_loader, device=device)

NameError: name 'train' is not defined

In [ ]:
from matplotlib.pyplot import imshow, show
import numpy as np
imshow(x_train[0])
show()

In [5]:
from src.utils import conf_matrix
conf_matrix(val_dataset, results)

NameError: name 'results' is not defined

In [3]:
unet.load_state_dict(torch.load('/Users/laszlofreund/code/ai/satellite-image-segmentation/unet.h5', map_location=device))

<All keys matched successfully>

In [ ]:
eval = unet.evaluate(val_loader, device=device)